In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [6]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xbatch, ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [7]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [8]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [534]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, input_shape=input_shape))
model.add(keras.layers.LeakyReLU(0.2))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.PReLU())
model.add(keras.layers.Conv2D(32, 1))
model.add(keras.layers.LeakyReLU(0.2))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.PReLU())
#model.add(keras.layers.Conv2D(16, 1, activation='relu'))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(16, 3))
model.add(keras.layers.LeakyReLU(0.0))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.PReLU())
model.add(keras.layers.Conv2D(8, 5))
model.add(keras.layers.LeakyReLU(0.0))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.PReLU())
model.add(keras.layers.Conv2D(4, 3))
model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.PReLU())
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(2, 3))
#model.add(keras.layers.LeakyReLU(0))
#model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dropout(.4))
#model.add(keras.layers.Dense(16, activation='relu'))
#model.add(keras.layers.GaussianDropout(.5))
model.add(keras.layers.Dropout(.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [535]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [536]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_557 (Conv2D)          (None, 16, 16, 64)        8256      
_________________________________________________________________
leaky_re_lu_361 (LeakyReLU)  (None, 16, 16, 64)        0         
_________________________________________________________________
batch_normalization_451 (Bat (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_558 (Conv2D)          (None, 16, 16, 32)        2080      
_________________________________________________________________
leaky_re_lu_362 (LeakyReLU)  (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_452 (Bat (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_559 (Conv2D)          (None, 14, 14, 16)        4624      
__________

In [537]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=60, 
                    steps_per_epoch=64)

Epoch 1/60
64/64 [==============================] - 15s 228ms/step - loss: 0.6653 - acc: 0.6724 - val_loss: 0.5885 - val_acc: 0.7622
Epoch 2/60
64/64 [==============================] - 1s 23ms/step - loss: 0.5881 - acc: 0.7534 - val_loss: 0.5738 - val_acc: 0.7715
Epoch 3/60
64/64 [==============================] - 1s 23ms/step - loss: 0.5568 - acc: 0.7627 - val_loss: 0.5674 - val_acc: 0.7573
Epoch 4/60
64/64 [==============================] - 1s 23ms/step - loss: 0.5481 - acc: 0.7646 - val_loss: 0.5587 - val_acc: 0.7500
Epoch 5/60
64/64 [==============================] - 1s 23ms/step - loss: 0.5455 - acc: 0.7617 - val_loss: 0.5274 - val_acc: 0.7656
Epoch 6/60
64/64 [==============================] - 1s 23ms/step - loss: 0.5233 - acc: 0.7769 - val_loss: 0.5387 - val_acc: 0.7642
Epoch 7/60
64/64 [==============================] - 1s 23ms/step - loss: 0.5130 - acc: 0.7725 - val_loss: 0.5285 - val_acc: 0.7559
Epoch 8/60
64/64 [==============================] - 1s 23ms/step - loss: 0.4783 -

In [538]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(c, axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

ChatanoogaChooChoo 123
Predicted:  0.0018594408    Actual:  False 

BigJimMP 54
Predicted:  0.5219991    Actual:  False 

EternalFlame 123
Predicted:  0.13918093    Actual:  False 

BigJimMP 45
Predicted:  7.9061865e-05    Actual:  False 

GoodGollyMissMollyWithJerryLeeAndFats 55
Predicted:  0.033120293    Actual:  False 

BigJimMP 246
Predicted:  2.1529986e-10    Actual:  False 

IThinkWereAloneNow2MP 44
Predicted:  2.809149e-06    Actual:  False 

FugueForTinhorns2 41
Predicted:  0.37526393    Actual:  False 

IThinkWereAloneNow2MP 133
Predicted:  0.724471    Actual:  True 

HoldMeTight 108
Predicted:  0.21096328    Actual:  False 



<code>
with ,5 baseline:
activate after b/n -> 3054, 3095, 3618
leaky .1 -> 3149
prelu -> 3080, 3609
prelu d/o=.6 -> 3734
1-1-3-5-3-1 -> 2801, 3374, 3578
gaussian d/o -> 2911, 3217, 2953, **3097**, 3095, 3376, **3212**, 3427
1-1-3-5-5 -> 3215, 3180
leaky .2 on bottom 2 layers -> 2725, 3098, 2977, 3094, 2863, 3073, 3206, 3320, 3036, 
                             **3077**, 3217 SOMETIMES d/o=.6
leaky .2 on bottom 2 layers -> 3189, 2989, 3454    
leaky .4 on bottom 2 layers -> 3248
leaky .2/.1 on bottom layers -> 2839, 2832, **3135**, 2908, 3350, 3075, 3161, 3342, 3247
    
used .6 baseline
leaky .05 -> 3122, 3245
</code>

<code>
1-1-5-5-5 -> 3107, 3481 (median of last 5)
1-1-3-5-3 -> 3052, 3053, 2858, 3295, 3636, 2974, 3057, 2834, 3045, 3742, 3182, 
             3559, 3112, **3104**, 3148, 4013, 3305, 3015
1-1-3-5-3-1 -> 3064, 3129, 3062, 3237
1-1-3-5-3-1 d/o=.4 -> 2820, 3224, 3211
1-1-3-5-3 d/o=.45 -> 3215
1-1-3-5-3 d/o=.7 -> 3408
1-1-3-5-3-5 -> 3625
*1-1-3-5-3 d/o=.5 -> 3021, 3100, 2908, 3027, 2970, 3188, 3044, 3055, 3021, 3143, 3278, 
                     3378, 3213, **3058**, 2873, 2788, 3144, 2777, 3248, 2975, 3244, 
                     3219, 3570,  * 
1-1-3-5-3-3 d/o=.5 -> 3151, 3149, 3533
1-1-3-5-3 add maxpool d/o=.5 -> 3193
1-1-3-5-3 add maxpool d/o=.65 -> 4213
1-1-1-3-5-3 d/o=.5 -> 3204
</code>